# BEFORE YOU START
- Below is a template of the notebook in which demonstrate your solution.
- The notebook should be a stand-alone entity, meaning that anyone with basic CS and AI / ML knowledge who reads it should be able to understand your problem.
- The notebook must be in regular notebook format, e.g., you may not use RISE.
- This template is a guideline, and you may make any changes required to best present your problem and solution. **HOWEVER**, you must include all the information that is required of you in this template. Read the text and comments in the different sections to find out what these requirements are.
- Cite / link your sources, including all papers, blogs, and images taken from the web.
- Sections that require code have code cells with coding todos for that section.
- You are not required to stick to the placement of the code. It is preferred to sprinkle many short code cells within textual context than to put all code together at the end of the section (much more readable)
- Remember to change the project title and author lines!
- Do not include this section in your final submission (delte this cell)

# Settelers of catan inspired by Alphazero approach
Aviv Cohen; avivcohen@campus.technion.ac.il

Dan Navon; danavon@campus.technion.ac.il

## Introduction

An introduction should include the following:
- An informal description of the problem you are trying to solve (examples are best)
- Motivation for solving this problem
- An explanation why it is hard to solve this problem
- Previous methods for solving the problem and their strengths and weaknesses
- A description of your solution to the problem, how it overcomes the issues in previous mehtods, and what new issues arise.
- A short description on how you intend to evaluate your solution.

## Previous methods:
The ‘world champion’ standard of AI for this board game is a program called JSettlers, developed by Jeremy Monin over 10 years ago and still in active development today. This program is developed in Java and contains a fully-implemented framework of the board game as well as an entirely rule-based agent with hard-coded decisions and heuristics.

Previous research was endeavored by the QSettlers team led by Peter McAughan in 2019. In this work, the team attempted to similarly apply the DQN paradigm to develop an AI model to play and win Settlers of Catan.
Unfortunately, the team was unsuccessful at implementing a working DQN model for general gameplay, but was able to train and develop a working DQN model specifically for the player trading mechanism of the game.
        
At [Re-L Catan: Evaluation of Deep Reinforcement Learning for
Resource Management Under Competitive and Uncertain
Environments](https://cs230.stanford.edu/projects_fall_2021/reports/103176936.pdf) the authors uses Deep Q-Learning on different part of the game in order to the determine if this is the right action to do (e.g. building/trading) but they ignore the whole gameplay.

#TODO:
Game strategis 

[Optimizing UCT for Settlers of Catan](https://www.sbgames.org/sbgames2017/papers/ComputacaoFull/175405.pdf) based on [Monte-Carlo Tree Search in Settlers of Catan](https://www.researchgate.net/publication/220716999_Monte-Carlo_Tree_Search_in_Settlers_of_Catan)

[Mastering the game of Go with deep neural networks and tree search](https://www.nature.com/articles/nature16961.pdf)


In [ ]:
# TODO: provide a step-by-step guide to running your domain in code
#  - initialize an environment instance
#  - visualize observations ()
#  - show how to take an action.
#  - play a game / an episode / several timesteps

## Domain - Catan

We forked our project from [PyCatan2](https://github.com/josefwaller/PyCatan2), which gives a raw implmentation of the game in order to let other developers to implment the rules they inted to use.
In our case, we implmented the very basic game elements - building cities, settelments, roads and trading (no players' trading).
The reason is to keep the game state fully visible as in other AlphaZero games implementation (Go, Chess, 4-in a row, tic-tac-toe and etc) so we can be compatible with the original game.

Full explanation of the game can be found [here](https://en.wikipedia.org/wiki/Catan) or under docs directory

In [1]:
import os.path
import random
import torch
import matplotlib.pyplot as plt

from src.mcts import mcts_get_best_action
from src.mlp import MLP
from src.dataset import Dataset
from src.training import MLPTrainer
from src.plot import plot_fit


We wrapped the PyCatan2 in order to let the agents interact with game.

One can change <code>catan_wrp.py</code> , add or remove different rules of the game, change the initial board and etc.

In [2]:
from src.catan_wrp import Catan
num_players = 4
catan_game = Catan()
print("Board:")
print(catan_game.game.board)

Board:
                                                       
                                                       
                                                       
                                                       
                 3:1         2:1                       
                  .--'--.--'--.--'--.                  
                  | 10  |  2  |  9  | 2:1              
               .--'--.--'--.--'--.--'--.               
           2:1 | 12  |  6  |  4  | 10  |               
            .--'--.--'--.--'--.--'--.--'--.            
            |  9  | 11  |   R |  3  |  8  | 3:1        
            '--.--'--.--'--.--'--.--'--.--'            
           2:1 |  8  |  3  |  4  |  5  |               
               '--.--'--.--'--.--'--.--'               
                  |  5  |  6  | 11  | 2:1              
                  '--.--'--.--'--.--'                  
                 3:1         3:1                       
                                         

The coordination system called is a skwed 2D grid, more details can be found under <code>/docs/Working-with-Board.srt</code>


<img src="docs/images/catangrid_withpieces.png" align="left"/>
<img src="BeginnerBoard.jpg" align="right"/>

### Observations
The observations returned as a seralized vector.
The reason for this serialization is that we would like to predict how good a state is using a DNN which have to recive a constanst size input vector.

observations represent = <code>[current_player, dice, longest_road, initializtion_stage, intersection_buildings, roads, resources, harbors] </code>.
1. <code>current_player</code> - current player id.
2. <code>dice</code> - dice value can be ranged between 2-12.
3. <code>longest_road</code> - the player id who owns the longest road (2 victory points).
4. <code>initializtion_stage</code> - boolean value, indicates if we are at the initialization stage.
5. <code>intersection_buildings</code> - vector with 55 elements where values represents the 55 intersections along the board. values repesented as $player\_id*10 + building\_type$ where $building\_type$ may be- 1-Settlement, 2-City. if no bulding han the value is - 0 - No building.
for example the value "12" means that the intersection belongs to player_id==int(12/10)==1 with a city(12%10==2) on it.
6. <code>roads</code> - vector with 71 elements - represnts all posiible locations to place roads at. values may be - 0 - No road, otherwise - player_id+1 where the player_id represnt who onws this road.
7. <code>resources</code> - there are 5 different types of resources (lumber, brick, ore, grain, grass), assuming 4 players in the game, this vector will have 20 elments where each elments represnts the amount of each resource
8. <code>harbors</code> - 9 elements, the values are as in intersection_buildings, harbors change the trading rates from 4/1 to 3/1
 
overall the state is reprensted in a compcat 160 essential elements.

at this image, green dots represnting the intersection, between each 2 adjacent dots, a road can be placed

<img src="intersections.png"/>

Image taken from ["RE-L Catan"](https://cs230.stanford.edu/projects_fall_2021/reports/103176936.pdf)


In [3]:
state = catan_game.get_state()
print("state size:" + str(len(state)))
print(state)

state size:160
tensor([0., 8., 0., 0., 2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])


### Actions
The actions space contains - building(city / settelement/ road), trading or end turn(+dice roll)
Not all the action always avilable, the building and trading avilabilty depends on the number of resources each player has

The first 8 turns (assuming 4 players) are being played in a different way.
The order of the players is 1-2-3-4-4-3-2-1.
This stage is the picking stage where each player pick a settlement and a road. at the reverse order (4-3-2-1) the players recive resources as well according to the surround hexes. 
Becuase each player can pick any leagal settlement at the beggining, the number of actions can be up to 55.

Each action is represented as a tuple where the first element reflects the type of action and the second the coordinaton of the action.

0. build a road
1. build a settlement
2. build a city
3. trade resources (second value is the type of the trade)
4. end turn

In [44]:
actions = catan_game.get_actions(prune=False)
for a in actions:
    print(a)
best_action = random.choice(actions)
catan_game.make_action(best_action)
print(catan_game.game.board)

(<BuildingType.ROAD: 0>, frozenset({(q: 0, r:-4), (q: -1, r:-3)}))
(<BuildingType.ROAD: 0>, frozenset({(q: -2, r:0), (q: -2, r:-1)}))
(<BuildingType.ROAD: 0>, frozenset({(q: 0, r:-1), (q: 0, r:-2)}))
(<BuildingType.ROAD: 0>, frozenset({(q: -3, r:2), (q: -4, r:3)}))
(<BuildingType.ROAD: 0>, frozenset({(q: 1, r:-3), (q: 0, r:-2)}))
(<BuildingType.ROAD: 0>, frozenset({(q: -5, r:2), (q: -5, r:3)}))
(<BuildingType.ROAD: 0>, frozenset({(q: -3, r:-1), (q: -2, r:-1)}))
(<BuildingType.ROAD: 0>, frozenset({(q: -4, r:4), (q: -4, r:3)}))
(4,)
                                                       
                                                       
                                                       
                                                       
                 3:1         2:1                       
                  .--'--.--'--.--'--.                  
                  | 10  |  2  |  9  | 2:1              
               s--'--.--'--.--'--.--'--.               
           2:1 

### Game simulation
Here are a few moves to adress how the game is progressing.
One can use the <code>src/text_game.py</code> in order to play by himself

In [ ]:
for i in range(40):
    actions = catan_game.get_actions(prune=False)
    action = random.choice(actions)
    catan_game.make_action(action)
    print("Player " + str(catan_game.get_turn() + 1) + ", action:" + str(action))

print(catan_game.game.board)

## Model

// _describe in detail how your model of the above domain, including objective (e.g. reward function), dynamics (e.g., transition function), states, observations, actions, etc. (refer to the lecture [Models in AI](https://moodle.technion.ac.il/pluginfile.php/1831187/mod_resource/content/2/236606_ModelsInAI.pdf))_

We model the environment as an MDP

$\mathfrak{X}$ - $\{[current_player, dice, longest_road, initializtion_stage, intersection_buildings, roads, resources, harbors]\}$
The state space details can be found under Observations section

${\mathfrak{A}} = \{\mathcal{A}^i\}_{i=1}^{4}$  where $\mathcal{A}^i(s)$ = \{"possible trades","possible buildings(cities/settlements/roads)", "end turn"\}
$\newline$ The action space details can be found under Action section. The trading and building possibilities depends on the resources availability. 

$\mathfrak{P} (s'|s,a)$ - The probability of getting the state s', when we are in the state s, and make the action a.
The stochastic transition happens only when $s.phase = "dice" $.
The stochasticity is over $s'.resources$ for all players due to a new resources allocation (see the [rules](https://en.wikipedia.org/wiki/Catan) of the game)
     
$\mathfrak{R} = \{\mathcal{R}^i\}_{i=1}^{4}$ - reward functions. $\mathcal{R}^i=\frac{agent^i\_ score}{\sum{scores}}$ 
    
Our objective is to find an optimal policy for our agent that maximizes his utility function.


## Solution

1. _describe theoretically how you are solving the problem. include here any guarantees (and their proofs) that your solution provides_
2. _describe practically your method solving the problem_
3. _describe any implementation challenges you faced_


<img src="images/plan.png" width="900" />

The planning diagram



<b>How we solved the problem:</b>
Our solution is a combination between a variation of MCTS, and DNN.
The DNN role:
The roll of the DNN is to estimate the value function (the result of the game).
It takes a vectorized representation of a game's state as an input, and returns a prediction of all the players' results at the end of the game, when the game stats from the given state.
Each training game, we create a dataset with all the states we visited along the game, and with the result of the game as the same label of each sample.
At the end of each game, we train the DNN to improve its predictions in the next game.

The MCTS role:
Each train along the training game, we use a variation of MCTS to choose the action of the current player.

The variation of the MCTS:
Contrary to the regular MCTS, our MCTS consists of more than one player, as mentioned in the diagram.
Each layer of the MCTS, belongs to other player, and the order of the players determined according to their order in the game.
In addition, instead of states nodes only, our MCTS contains actions nodes too.

The selection phase:
The selection phase in our MCTS, chooses the next node according to the UCT criterion, but from the current player perspective. That is, different players will get different UCT values of the same node.
In our MCTS, the UCT value will be calculated by the sum of the rewards of the CURRENT PLAYER.

The insertion phase:
Contrary to the regular MCTS, our version inserts not only the new state, but also the possible actions from it.

The simulation phase:
In our version, instead of the simulation phase, we use the DNN to predict the result of the game from the current state.

The propagation phase:
The propagation phase of our version will look the same as the original one, but include the updates of the action nodes.
Each action node, will get the same value of its parent state node.

The guarantees of our model:
Because we rely on the DNN to predict the result of the game, the results of our algorithm depend on the inclusion ability of the DNN, so there is no guarantees about the executions of our algorithm.


<b>How practically our method solves the problem:</b>
As we mentioned above, the roll of the DNN is to learn from sets of states and the end results.
If the inclusion ability of the DNN will be satisfying, it will improve its predictions every game, and better actions will be taken by the MCTS.


<b>Implementation challenges:</b>
* In the original version of the UCT, we should give the highest priority to non visited nodes. Catan game has a big branching factor, and it would take a lot of time to explore all the unvisited nodes in the tree. we faced with this issue by using some pruning that will shrink the exploration part of the UCT, to make it possible with our hardware limitations.

* At the beginning of the training, the DNN initialized with random weights. so all the players took random actions along the first game, and close to random actions in the next few games. We found out that there is a big chance to run the game forever when all the players play randomly. We faced this issue by adding a heuristic function to the DNN result, to make the trains a little more sophisticated, to make the first games done.

* When the selection phase in the MCTS chooses to take some action, we have to know what will be the given state after this action.
In Catan, the given state depends on the cubes result after making an action, so we couldn't know what will be the given action.
We couldn't choose one of the possible states randomly, because their distribution isn't uniform.
We faced with this issue by simulating the chosen action, and taking the given state in the simulator. That way it will converge to the right distribution of the possible states,


TODO(Dan): We can look at the git to see what has been changed

In [ ]:
# TODO: show core functions and classes for your solution
#   - must be clean, tidy, and well documented code.
#   - do not add basic tools and utilities. put those in your code base and import.
#   - you may display your code in markdown instead if you don't wish to run it,
#     and prefer to import it from your code base

# TODO: add MCTS main functions with comments

In [ ]:
# TODO: show a running example of your method
#   - run algorithm (e.g. training) or load pre-calculated values (e.g. model weights) 
#   - visualize run (if applicable)
#   - show results for this run
#   - TIP: use a constant random seed to ensure deterministic results

In [45]:
"""
Parameters definition
"""
# training parms
hp_model_training = dict(loss_fn=torch.nn.MSELoss(),
                         batch_size=100,
                         num_epochs=100,
                         test_ratio=0.2,
                         valid_ratio=0.2,
                         early_stopping=100)
# optimizer params
hp_optimizer = dict(lr=0.001,
                    weight_decay=0.01,
                    momentum=0.99)
# NN structure params
hp_model = dict(hidden_layers_num=1,
                hidden_layers_size=20,
                activation='relu')

#MCTS params: c - UCT exploration/exploitation param, d-weight on heuristic importance against the model(NN)
hp_mcts = dict(c=1,
               d=3,
               iterations_num=200)


In [6]:
# NN creation
def create_model(in_dim, out_dim, model_file):
    if os.path.isfile(model_file):
        print(f'loading model from "{model_file}"...')
        mlp = torch.load(model_file)
        print(mlp)
        return mlp

    mlp = MLP(
        in_dim=in_dim,
        dims=[hp_model['hidden_layers_size']] * hp_model['hidden_layers_num'] + [out_dim],
        nonlins=[hp_model['activation']] * hp_model['hidden_layers_num'] + ['none']
    )

    print('creating model...')
    print(mlp)
    return mlp


# training function
def train(dl_train, dl_valid, dl_test, model):
    loss_fn = hp_model_training['loss_fn']
    optimizer = torch.optim.SGD(params=model.parameters(), **hp_optimizer)
    trainer = MLPTrainer(model, loss_fn, optimizer)

    return trainer.fit(dl_train,
                       dl_valid,
                       num_epochs=hp_model_training['num_epochs'],
                       print_every=10,
                       early_stopping=hp_model_training['early_stopping'])

In [46]:
"""
Agent holds the model it will be used to make decisions under the MCTS.
Each agent can be use different model to predict the value function on a given state.
"""
class Agent:
    def __init__(self, model, prune=True):
        self.model=model
        self.prune=prune #boolean parmeter

In [8]:
def train_agent(games_num, model_path):
    model = create_model(Catan.get_state_size(), Catan.get_players_num(), model_path)
    agents = Catan.get_players_num()*[Agent(model)]

    for i in range(1, games_num + 1):
        print(f'_________________game {i}/{games_num}________________')

        catan_game = Catan()

        ds = Dataset(hp_model_training['batch_size'], hp_model_training['valid_ratio'], hp_model_training['test_ratio'])

        turns_num = 0
        actions_num = 0

        while True:
            actions_num += 1

            best_action = mcts_get_best_action(catan_game, agents, hp_mcts['c'], hp_mcts['d'], hp_mcts['iterations_num'])
            print("Player " + str(catan_game.get_turn() + 1) + ", action:" + str(best_action))

            reward = catan_game.make_action(best_action)
            if best_action[0] == 4:
                turns_num += 1

                print("Player " + str(catan_game.get_turn() + 1) + " turn!, dice: " + str(catan_game.dice))
                # print(catan_game.game.board)

            ds.add_sample(catan_game.get_state())
            if catan_game.is_over() or actions_num > 600:
                if actions_num > 600:
                  print("No winner, Final board:")
                  print(catan_game.game.board)
                  reward = [catan_game.game.get_victory_points(catan_game.game.players[i]) for i in range (Catan.get_players_num())]
                else:
                  print("Congratulations! Player %d wins!" % (catan_game.cur_id_player + 1))
                  print("Final board:")
                  print(catan_game.game.board)


                ds.set_label(reward)

                dl_train, dl_valid, dl_test = ds.get_data_loaders()
                fit_res = train(dl_train, dl_valid, dl_test, model)
                plot_fit(fit_res, log_loss=False, train_test_overlay=True)
                plt.show()
                print(ds)

                print(f'saving model in "{model_path}"')
                torch.save(model, model_path)
                break


In [ ]:
train_agent(2, "src/model2")

loading model from "src/model2"...
MLP(
  (mlp_layers): Sequential(
    (0): Linear(in_features=160, out_features=20, bias=True)
    (1): ReLU()
    (2): Linear(in_features=20, out_features=4, bias=True)
    (3): Identity()
  )
)
_________________game 1/2________________


## Evaluation

_define the evaluation metrics used to measure the sucess of your experiments. Explain why they are relevant for your problem_

1. After playing number of games, we trained our neural network and then run multiple games in order to collects statistics over agents' winning rate.
2. The evaluation has been done by comparing different types of agents:<br>
     a. $Agent 1$ - Trained NN with pruning.<br>
     b. $Agent 2$ - Trained NN without pruning.<br>
     c. $Agent 3$ - No NN with pruning.<br>
     d. $Agent 4$ - No NN without pruning.<br>
     c. $Agent 5$ - Random actions.<br>

* Agents 1-4 is based on the MCTS, with 300 iterations, using heuristic.


In [ ]:
def test_agent(games_num):
    model1 = create_model(Catan.get_state_size(), Catan.get_players_num(), 'model')
    model2 = create_model(Catan.get_state_size(), Catan.get_players_num(), 'src/model2')
    agents = [Agent(model1,prune=False), Agent(model1,prune=False), Agent(model1), Agent(model2)]
    stats = {k: [] for k in range(Catan.get_players_num())}
    for i in range(1, games_num + 1):
        print(f'_________________game {i}/{games_num}________________')
        catan_game = Catan()
        actions_num = 0
        turns_num = 0
        while True:
            actions_num += 1
            if catan_game.get_turn() == 0:
                actions = catan_game.get_actions(prune=False)
                best_action = random.choice(actions)
                # reward = catan_game.make_action(tuple(best_action))

            else:
                best_action = mcts_get_best_action(catan_game, agents, hp_mcts['c'], hp_mcts['d'], hp_mcts['iterations_num'])

            print("Player " + str(catan_game.get_turn() + 1) + ", action:" + str(best_action))
            reward = catan_game.make_action(best_action)
            if best_action[0] == 4:
                turns_num += 1
                print("Player " + str(catan_game.get_turn() + 1) + " turn!, dice: " + str(catan_game.dice))
                # print(catan_game.game.board)

            if catan_game.is_over() or actions_num > 600:
                if actions_num > 600:
                  print("No winner, Final board:")
                  print(catan_game.game.board)
                else:
                  stats[catan_game.cur_id_player].append([actions_num, int(turns_num/4)])
                  print("Congratulations! Player %d wins!" % (catan_game.cur_id_player + 1))
                  print("Final board:")
                  print(catan_game.game.board)
                break
    return stats

## Results

1. _describe what you expected to see in your results._
2. _describe your results and cmopare them to your expectations._

In [49]:
num_of_tests = 4
winning_records = test_agent(num_of_tests)
stats= [(100*len(v)/num_of_tests) for k,v in winning_records.items()]
print("winning rate(prectnage)")
print(stats)
print with agents description

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(with agents description)? (383974473.py, line 6)

In [48]:
winning_records = {0: [], 1: [], 2: [[156, 21], [216, 26]], 3: []}
stats= [(100*len(v)/2) for k,v in winning_records.items()]
print("winning rate(prectnage)")
print(stats)

winning rate(prectnage)
[0.0, 0.0, 100.0, 0.0]


In [ ]:
# TODO: visualize reults
#   - tables of evaluation method
#   - compare to previous works (if results available)
#   - any other graphs / charts / plot that visualize your method's performance

## Method Limitations / Possible Future Extensions

_describe where any limitations or drawbacks to your method, as well as any suggestions for future work that will improve upon it._

we need a DQN for the initial placement stage.
No utilization of the full game, the settings are pretty shallow
no exact strategy
didn't test it against humam player or other methods such as Jsettlers

In [ ]:
# TODO: visualize reults
#   - tables of evaluation method
#   - compare to previous works (if results available)
#   - any other graphs / charts / plot that visualize your method's performance

## Method Limitations / Possible Future Extensions

_describe where any limitations or drawbacks to your method, as well as any suggestions for future work that will improve upon it._

we need a DQN for the initial placement stage.
No utilization of the full game, the settings are pretty shallow
no exact strategy
didn't test it against humam player or other methods such as Jsettlers

In [ ]:
# TODO: show a running example that illustrates the above limitations